In [1]:
from mistralai import Mistral
from pdfminer.high_level import extract_text
import os
from dotenv import load_dotenv
import csv

In [2]:
def extract_text_pdf(pdf_path):
    text = extract_text(pdf_path)
    return text

In [3]:
pdf_path = '../data/pdf/college_resume.pdf'
print(extract_text_pdf(pdf_path))

AAYUSH KUMAR 
Address: Rishi comforts PG 578, 
60 Feet Rd, AECS, Layout - C Block, 
Marathahalli, Bengaluru Karnataka - 560037 
Email ID: aaku21ainds@cmrit.ac.in 
Mobile No: +91 895 593 1443 
LinkedIn : https://www.linkedin.com/in/aayush-kumar-6b5191263/ 
GitHub: Aayush-93407 
 CAREER OBJECTIVE 
Dynamic  and  driven  Bachelor  of  Engineering  graduate  from  CMR  Institute  of  Technology,  specializing  in  Artificial 
Intelligence and Data Science, seeking a challenging role where I can leverage my strong analytical skills and passion 
for  innovation.  Committed  to  continuous  learning  and  professional  growth,  drive  efficiency,  and  create  value  in  a 
dynamic and collaborative environment. 
 EDUCATION QUALIFICATION 
•  Bachelors of Engineering – Artificial Intelligence and Data Science Engineering 

CMR Institute of Technology, Bengaluru 
9.03 CGPA, 2025(Pursuing) 

•  12th Grade – Science 

Central Academy, Kota, Rajasthan 
79.6%, 2021 
•  10th Grade 

Excellent Public 

In [4]:
def analyze_resume(text, api_key_mistral):
    model = "mistral-large-latest"
    prompt = f"Analyze this text from a resume and return the following information on new lines \nName, Email ID, Contact Number, LinkedIn and GitHub page links, and extract all skills from projects and those under the skills section and pack them all under the skills field, seperated by commas \nGive me these details in the form field_name : values, each on a new line\nPlease make sure the field names are exactly like this: Name, Email ID, Contact Number, LinkedIn, GitHub, Skills\nPlease don't include any other unnecessary sentence in your response \n{text}"
    
    client = Mistral(api_key=api_key_mistral)
    
    completion = client.chat.complete(
        model=model,
        messages=[
            {
                "role":"user",
                "content": prompt
            }
        ]
    )
    return completion.choices[0].message.content

In [5]:
def recommend_projects(skills, api_key_mistral):
    model = "mistral-large-latest"
    prompt = f"Based on the skills provided below, recommend top 5 projects that the user can work on and successfully complete\nGenerate your response in this format: Project Name, Project Description, Tools and Skills Utilized, and Steps \nPlease make sure you don't include unnecessary sentences in your response \n{skills}"
    
    client = Mistral(api_key=api_key_mistral)
    
    completion = client.chat.complete(
        model=model,
        messages = [
            {
                "role":"user", 
                "content":prompt
            }
        ]
    )
    
    return completion.choices[0].message.content

In [6]:
def create_documentation(project_desc, api_key_mistral):
    model = "mistral-large-latest"
    prompt = f"Based on the project description given below, create a well-structured markdown documentation suitable for the project's Github repository. The documentation should display the tools used, a detailed project description, and setup and execution steps \nPlease make sure you add no unnecessary sentences in your response \n{project_desc}"
    
    client = Mistral(api_key=api_key_mistral)
    
    completion = client.chat.complete(
        model=model,
        messages = [
            {
                "role":"user", 
                "content":prompt
            }
        ]
    )
    
    return completion.choices[0].message.content

**MAIN FOR RESUME FIELD EXTRACTION**

In [11]:
def main():
    filepath = '../data/pdf/1CR21EC252_ADARSHVINOD_RESUME.pdf'
    
    ## Generate API key and store in .env file
    # Load Mistral API key
    load_dotenv("C:\\Users\\ADMIN\\Desktop\\api_key.env")
    api_key_mistral = os.getenv("MISTRALAI_API_KEY")
    
    if os.path.splitext(filepath)[1] == '.pdf':
        resume_text = extract_text_pdf(filepath)
        
    user_info = analyze_resume(resume_text, api_key_mistral)
    # user_info -> string
    
    user_info_dict = {}
    # split by \n
    lines = user_info.split("\n")
    for line in lines:
        key, *value = line.split(':', 1)
        if not value:
            continue
        user_info_dict[key.strip()] = value[0].strip()
        
    # print(user_info_dict)
        
    # user_info_dict = {
    #     "Name": name,
    #     "Email ID": emails[0] if emails else "",
    #     "Contact Number": phone_numbers[0] if phone_numbers else "",
    #     "LinkedIn": next((link for link in links if 'linkedin' in link), ""),
    #     "Github": next((link for link in links if 'github' in link), ""),
    #     "Skills": ', '.join(skills)
    # }
    
    csv_filename = "user_info.csv"
    with open(csv_filename, mode="a", newline="") as file:
        fieldnames = ["Name", "Email ID", "Contact Number", "LinkedIn", "GitHub", "Skills"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        if file.tell() == 0:
            writer.writeheader()
            
        writer.writerow(user_info_dict)
        
        file.close()
        
    # Project recommendations
    # projects -> string
    projects = recommend_projects(user_info_dict["Skills"], api_key_mistral)
    
    # Project documentation generation
    projects_split = projects.split(". **")
    print(f"projects_split: {projects_split}")
    
    project_index = 3
    if project_index % 2 == 0:
        project_index = project_index+1 if project_index + 1 < len(projects_split) else project_index-1
        
    project_documentation_reponse = create_documentation(projects_split[project_index][:-1], api_key_mistral)
    print(project_documentation_reponse)
    
    md_filename = "README.md"
    with open(md_filename, mode="w") as file:
        file.write(project_documentation_reponse)
        file.close()
    

In [12]:
main()

projects_split: ['1', 'Smart Irrigation System**\n   - **Project Description:** Develop an IoT-based smart irrigation system that monitors soil moisture and weather data to automate watering.\n   - **Tools and Skills Utilized:** Arduino, ESP32, Python, SQLite3, Git, OpenCV, Machine Learning\n   - **Steps:**\n     1. Set up Arduino and ESP32 for data collection.\n     2. Use OpenCV for image processing to monitor soil conditions.\n     3. Implement machine learning algorithms for predictive analytics.\n     4. Store data in SQLite3.\n     5. Use Git for version control.\n\n2', 'Interactive Web Dashboard**\n   - **Project Description:** Create a web dashboard that displays real-time data and allows user interaction.\n   - **Tools and Skills Utilized:** HTML, CSS, JavaScript, ReactJS, Django, Bootstrap, NewsAPI, MailTrap\n   - **Steps:**\n     1. Design the frontend using HTML, CSS, and Bootstrap.\n     2. Implement the backend with Django.\n     3. Use ReactJS for interactive components.